In [2]:
import ee
import numpy as np
import folium
import json
from util import shp_to_ee_fmt,add_ee_layer,get_whole_region
import time 
from geetools import tools
import geopandas as gpd
import pandas as pd
from yearly_bar_plot_tree_cover import pixel_stats_dynamic_world,plot_multiple_bar_from_dict,pixel_stats_world_cover,pixel_stats_global_forest_watch
from constants import * 
import matplotlib.pyplot as plt


folium.Map.add_ee_layer = add_ee_layer    

try:     
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

/Users/gclyne/.pyenv/versions/3.9.1/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
# TO EXPROT CSV OF YEAR FOR EACH REGION WITH WC


shape_file = '/Users/gclyne/Downloads/rfcx_sensor_placement/Graham_Amazon/Bacajai_JP1.shp'
file_name = shape_file.split('/')[-1].split('.')[0]
start_date = '2017-01-01'
end_date = '2021-01-01'
shp_file = gpd.read_file(shape_file, crs='EPSG:4326')
trees = []
data = json.loads(shp_file.to_json())

    #for each feature in image, get monthly avg and get band values
for index in range(len(data['features'])):
    print(f'processing image: {index}')
    # #get image of region
    region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,index))

    dataset = ee.ImageCollection("ESA/WorldCover/v100").first().clip(region)
    pixelCountStats = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=10)
    total = sum(pixelCountStats.getInfo()['Map'].values())

    output = pixelCountStats.getInfo()['Map']['10'] / total
    # for key,value in data.items():
    #     print(key,value)
    #     sum = 0
    #     for val in value['label'].values():
    #         print(val)
    #         sum = sum + val
    #     new_dict[key] = {'1':value['label']['1'] / sum}
    # # print(new_dict)
    # datas.append(new_dict)
    trees.append(output)
df_wc = pd.DataFrame({'id':range(len(data['features'])),'2020':trees})
df_wc
df_wc.to_csv('wc_2020_tembe.csv')

# pixelCountStats = dw.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=1)
# output = pixelCountStats.getInfo()
# print(output)
#

In [ ]:
#make CSV for GFC 

shape_file = '/Users/gclyne/Downloads/rfcx_sensor_placement/Graham_Amazon/Bacajai_JP1.shp'
file_name = shape_file.split('/')[-1].split('.')[0]
start_date = '2017-01-01'
end_date = '2021-01-01'
shp_file = gpd.read_file(shape_file, crs='EPSG:4326')
trees = []
data = json.loads(shp_file.to_json())

    #for each feature in image, get monthly avg and get band values
for index in range(len(data['features'])):
    print(f'processing image: {index}')
    # #get image of region
    region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,index))

    dataset = ee.ImageCollection("UMD/hansen/global_forest_change_2021_v1_9").first().clip(region)
    pixelCountStats = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=10)
    total = sum(pixelCountStats.getInfo()['Map'].values())

    output = pixelCountStats.getInfo()['Map']['10'] / total
    # for key,value in data.items():
    #     print(key,value)
    #     sum = 0
    #     for val in value['label'].values():
    #         print(val)
    #         sum = sum + val
    #     new_dict[key] = {'1':value['label']['1'] / sum}
    # # print(new_dict)
    # datas.append(new_dict)
    trees.append(output)
df_wc = pd.DataFrame({'id':range(len(data['features'])),'2020':trees})
df_wc
df_wc.to_csv('wc_2020_tembe.csv')

In [ ]:
import plotly.express as px
fin_df = fin_df.reset_index()
px.bar(fin_df,x='id',y=['2017','2018','2019','2020'],barmode='group')

In [ ]:
shape_file = '/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp'
shp_file = gpd.read_file('/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp', crs='EPSG:4326')
file_name = shape_file.split('/')[-1].split('.')[0]  

my_map = folium.Map(location=[region.getInfo()['coordinates'][0][0][1],region.getInfo()['coordinates'][0][0][0]], zoom_start=13, height=500)
region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,1))

years = ['2017','2018','2019','2020','2021']
for year_index in range(4):
    dw = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(years[year_index],years[year_index+1]).filterBounds(region).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    classification = ee.Image(dw.first()).clip(region) #here we use mode reduction, takes the most common pixel value over time period
    s2VisParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}    
    my_map.add_ee_layer(classification, s2VisParams, 'Classified Image')
    my_map.save(file_name + str(year_index) + '_s2_copernicus_first_single_dw.html')



In [ ]:
shape_file = '/Users/gclyne/Downloads/Tembe_shp/Tembe_Reserve.shp'
shp_file = gpd.read_file('/Users/gclyne/Downloads/Tembe_shp/Tembe_Reserve.shp', crs='EPSG:4326')
folium.Map.add_ee_layer = add_ee_layer    
file_name = shape_file.split('/')[-1].split('.')[0]  

region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,0))


dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2019','2020').filterBounds(region).limit(10)
images = []
def listval(img, it):
    classification = img
    return ee.List(it).add(img)

result = dw.iterate(listval, ee.List([]))
dwVisParams = {'min': 0,'max': 8,'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A','#C4281B', '#A59B8F', '#B39FE1']}
# my_map.add_ee_layer(classification, dwVisParams, 'Classified Image')
# my_map.save(file_name + str(year_index) + '_mode_dw.html')
result.getInfo()

In [ ]:
result_list = result.getInfo()
print(dw.first().select('label').clip(region).name())
# print(ee.Dictionary(result_list[0]).toImage().name())
# # result_list = [dw.sort('system:time_start', False).first().clip(region)]
# for i in range(len(result_list)):
#     print(i)
#     # print(ee.Dictionary(result_list[i]).toImage())
#     dwVisParams = {'min': 0,'max': 8,'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A','#C4281B', '#A59B8F', '#B39FE1']}
#     my_map = folium.Map(location=[region.getInfo()['coordinates'][0][0][1],region.getInfo()['coordinates'][0][0][0]], zoom_start=13, height=500)
#     my_map.add_ee_layer(ee.Dictionary(result_list[i]).toImage().select('label'), dwVisParams, 'Classified Image')
#     my_map.save(str(i) + '_test_single.html')

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2020','2020').filterBounds(region).limit(10)

videoArgs = {
  'dimensions': 768,
  'region': region,
  'framesPerSecond': 7,
  'crs': 'EPSG:3857',
  'min': 0,
  'max': 8,
  'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A','#C4281B', '#A59B8F', '#B39FE1']}
dw.select('label').getFilmstripThumbURL(videoArgs)


In [13]:
months = pd.date_range('2017-01-01', '2021-01-01',freq='M',inclusive='left').tolist()
columns = list(map(lambda x:f'{x.year}-{x.month}',months))
columns

['2017-1',
 '2017-2',
 '2017-3',
 '2017-4',
 '2017-5',
 '2017-6',
 '2017-7',
 '2017-8',
 '2017-9',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-1',
 '2018-2',
 '2018-3',
 '2018-4',
 '2018-5',
 '2018-6',
 '2018-7',
 '2018-8',
 '2018-9',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-1',
 '2019-2',
 '2019-3',
 '2019-4',
 '2019-5',
 '2019-6',
 '2019-7',
 '2019-8',
 '2019-9',
 '2019-10',
 '2019-11',
 '2019-12',
 '2020-1',
 '2020-2',
 '2020-3',
 '2020-4',
 '2020-5',
 '2020-6',
 '2020-7',
 '2020-8',
 '2020-9',
 '2020-10',
 '2020-11',
 '2020-12']

In [ ]:
shape_file = '/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp'
shp_file = gpd.read_file('/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp', crs='EPSG:4326')
file_name = shape_file.split('/')[-1].split('.')[0]

my_map = folium.Map(location=[region.getInfo()['coordinates'][0][0][1],region.getInfo()['coordinates'][0][0][0]], zoom_start=13, height=500)
region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,20))


dw = ee.ImageCollection('ESA/WorldCover/v100').first().clip(region)
visualization = {'bands': ['Map']}
my_map.add_ee_layer(dw, visualization, 'Classified Image')
my_map.save(file_name + '_max_wc.html')

dw = ee.ImageCollection('ESA/WorldCover/v100').filterBounds(region).first().clip(region)
pixelCountStats = dw.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=1)
print(pixelCountStats.getInfo()['Map'])

In [17]:
import plotly.express as px
df = px.data.tips()
print(df)
fig = px.bar(df, x="sex", y="total_bill", color='time')
fig.show()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]


In [ ]:
shp_file = gpd.read_file('/Users/gclyne/Downloads/Tembe_shp/Tembe_Reserve.shp', crs='EPSG:4326')
region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,0))

probabilityBands = [
  'water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub',
  'built', 'bare', 'snow_and_ice'
]
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate('2019-01-01','2020-01-01').filterBounds(region).select(probabilityBands)

data = tools.imagecollection.getValues(
collection=dw,
geometry=region,
reducer=ee.Reducer.frequencyHistogram(),
scale=1,
maxPixels=1e8,
side='client',
bestEffort=True)

import plotly.express as px


In [ ]:
data

for x in data:
    print(data[x]['label'])
    xdatapandas = tools.imagecollection.data2pandas(data[x])
    print(xdatapandas)

In [16]:
shape_file = '/Users/gclyne/Downloads/Tembe_shp/Tembe_Reserve.shp'
start_date = '2021-01-01'
end_date = '2021-12-31'
shp_file = gpd.read_file('/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Tembe_Reserve_sites_Buffer1k.shp', crs='EPSG:4326')
shp_file_1 = gpd.read_file('/Users/gclyne/Downloads/Tembe_shp/Tembe_Reserve.shp',crs='EPSG:4326')
region_whole = get_whole_region(shp_file_1)
region_sites = get_whole_region(shp_file)
file_name = shape_file.split('/')[-1].split('.')[0]

features = ee.FeatureCollection('projects/ee-clynegraham/assets/Tembe_Reserve_sites_Buffer1k')
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(start_date,end_date).filterBounds(region_whole).select('label')
dwImage = ee.Image(dw.mode()).clip(region_whole) #here we use mode reduction, takes the most common pixel value over time period
classification = dwImage.select('label')
dwVisParams = {'min': 0,'max': 8,'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A','#C4281B', '#A59B8F', '#B39FE1'],'opacity':1}
my_map = folium.Map(location=[region_sites.getInfo()['coordinates'][0][0][0][1],region_sites.getInfo()['coordinates'][0][0][0][0]], zoom_start=13, height=500)

s2VisParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
my_map.add_ee_layer(classification, dwVisParams, 'Classified Image')

my_map.add_ee_layer(features,{},'1km buffers')
my_map.save(file_name + '_dw.html')

IMAGE HERE


In [ ]:
def gfc_yearly_csv(shp_file,start,end):
    region = ee.Geometry.Polygon(shp_to_ee_fmt(geodf,index))

    cc = ee.Number(10)
    gfc2021 = ee.Image('UMD/hansen/global_forest_change_2021_v1_9')
    canopyCover = gfc2021.select(['treecover2000'])
    canopyCover10 = canopyCover.gte(cc).selfMask()
    forestSize = canopyCover10.reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= region,
        scale= 1,
        maxPixels= 1e9,
        bestEffort=True)

    forest_size = list(forestSize.getInfo().values())[0]

    treeLoss = gfc2021.select(['lossyear'])
    losses = []
    gfc2021.select(['gain']);
    for i in range(1,int(ee.Date(end).format('YY').getInfo())):
    # for i in range(1,3):
        print(f'calculating year {i}')
        treeLoss01 = treeLoss.eq(i).selfMask()
        treecoverLoss01 = canopyCover10.And(treeLoss01).rename('loss' + str(i)).selfMask()
        lossSize = treecoverLoss01.reduceRegion(
            reducer= ee.Reducer.sum(),
            geometry=region,
            scale= 1,
            maxPixels= 1e9,
            bestEffort=True
        )
        losses.append(list(lossSize.getInfo().values())[0])
        print(losses)
    gains = gfc2021.select(['gain'])
    gains_summed = gains.reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= region,
        scale= 1,
        maxPixels= 1e9,
        bestEffort=True);
    gains = gains_summed.getInfo()['gain'] 
    losses = np.array(losses) 
    losses[0] = losses[0] - gains
    result = []
    for i in range(len(losses)):
        total_cover = (forest_size - sum(losses[0:i])) / forest_size 
        result.append(total_cover)
    pd.DataFrame(result).to_csv('gfc_stats.csv')